## Simple implementation of Principal component analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

X is the dataset we want to represent in some lower dimensional space. Each row of X represents a point in n-D space where n is the size of each row ( i.e. no of columns ). Here n = 3, in order to plot the numbers using pyplot. Our aim is to convert each 3-D point into 2-D point while preserving as much information as we can. In other words ** if we needed to convert this compressed 2-D representation into 3-D representation again, we should be as near to the real values as possible** i.e. the overall error must be as small as possible. Lossely speaking , we aim to minimize || g(C) - X ||<sub>F</sub> where C is compressed representation and g is the decoder function.and F represents Frobeous Norm.)

In [ ]:
X = np.random.rand(5000,3)
print(X)

Calculating the eigen vectors and eigen values of X<sup>T</sup>X .

In [ ]:
import numpy.linalg as LA
eigen_val,eigen_vec = LA.eigh(np.transpose(X) @ X)
print(eigen_val)
print(eigen_vec)

To decode data into 2-D space we want 2 eigen vectors corresponding to 2-largest eigen values. The eigen values produced by numpy.linalg.eigh are already in increasing order so we just choose the last 2 vectors from the list. D contains the required 2 vectors.

Note : All the eigen values are real because the X<sup>T</sup>X is a real symmetric matrix. This allows us to define the ordering .In addition to this the eigen vectors are orthonormal to each other ( again because the concerned matrix is a real symmetric matrix.) This satisfies the two condition mentioned in the [proof](https://www.deeplearningbook.org/contents/linear_algebra.html):

>    - All eigen vectors in matrix D should be orthogonal to each other.( D is not a square matrix so avoiding " D is orthogonal matrix")
>    - DD<sup>T</sup> = I

In [ ]:
D = eigen_vec[-2:]
print(D)

In [ ]:
print(X.shape)
print(D.shape)

In current numpy representation row represent an eigen vector ( A bit different from proof where column represent eigen vectors.) We define the Encoding function f and decoding function g ( @ represent matrix multiplication ):
>   - g(c) = c @ D ( Consider General D )
>   - f(x) = c = x @ D<sup>T</sup> ( Choose best c given D)
>   - D = { k eigen vectors corresponding to k-highest eigen values }  ( Choose D such that total error is minimum ).

Quoting from proof : 

>    "PCA is defined by our choice of the decoding function.Specifically, to make the
>    decoder very simple, we choose to use matrix multiplication to map the code back
>    into R<sup>n</sup> . Let g( c ) = Dc , where D ∈ R <sup>n×l</sup> is the matrix defining the decoding."

Notice the difference in order of multiplication. The given formula implements same things just in different order.

In [ ]:
C = X @ np.transpose(D)
print("------Encoded values ---\n",C)
X_approx = C @ D
print("------Decoded values ---\n",X_approx)

In [ ]:
print("------Encoded values ---\n",C)
print("------Decoded values ---\n",X_approx)

In [ ]:
class drawline():
    def __init__(self,P1,P2,axis,color='pink'):
        self.P1 = P1
        self.P2 = P2
        self.color = color
        self.axis = axis

    def draw(self):
        x = [self.P1[0],self.P2[0]]
        y = [self.P1[1],self.P2[1]]
        z = [self.P1[2],self.P2[2]]
        ax = self.axis
        c = self.color
        ax.plot3D(x,y,z,c=c)

## PCA gives us a direction to look along for truest picture.

The Graph above shows the working of PCA as follows : You encode the original 3-D data( red color ) into 2-D data ( green dots in graph below ). Now we wish to decode this encoded data again. In order to do so we are multiplying C by D where D consists of only 2 suitable eigen vectors each of length 3. ** The decoded values will all lie on a plane as the decoded points are just linear combination of these 2 eigen vectors **. So for n-D to k-D encoding we will need an optimal k-D plane to store the decoding and then we will generate an equivalent k-D plane(space) consisting of n-dimensional points ( Here we generated 2-D plane formed by 3-dimensional points ) and this k-D plane will be the truest picture we can obtain.

So, We now have the explanation to our initial example for points A(1,0,0) and B(2,0,0). We could have choosen any plane parallel to x-axis but not Y-Z. In fact the planes containing x-axis are the best as they will decode into exact same picture of the two points.

In [ ]:
from mpl_toolkits import mplot3d

# plt.rcParams['legend.fontsize'] = 25
fig1 = plt.figure(figsize=(10,10))
ax = plt.axes(projection = '3d')
ax.scatter3D(X[:,0],X[:,1],X[:,2],c='red',label='original')
ax.scatter3D(X_approx[:,0],X_approx[:,1],X_approx[:,2],c='blue',label='Decoded')
ax.legend()

i = 0
for x in X:
    x_approx = X_approx[i]
    i+=1
    line = drawline(x,x_approx,axis=ax)
    line.draw()
# plt.savefig("PCA_1.png")


In [ ]:
fig2 = plt.figure(figsize=(10,10))

# plt.rcParams['legend.fontsize'] = 15 
ax = plt.axes()
ax.scatter(C[:,0],C[:,1],c='green',label='Encoded')
ax.legend()
# plt.savefig("PCA_2.png")

## Visualizing Simple Geometric Shapes

In [ ]:
class PCA():
    def __init__(self,original,k_dim):
        self.X = original
        self.k_dim = k_dim
    
    def Encode(self):
        eigen_val,eigen_vec = LA.eigh(np.transpose(X)@X)
        self.D = eigen_vec[- self.k_dim :]
        self.C = self.X @ np.transpose(self.D)
        return self.C
    
    def Decode(self):
        X_approx = self.C @ self.D
        return X_approx
        

In [ ]:
class DrawResult():
    # import matplotlib as pyplot
    def __init__(self,original , encoded , decoded):
        self.X = original
        self.C = encoded
        self.X_approx = decoded
    def draw_3D(self):
        # plt.rcParams['legend.fontsize'] = 15
        fig1 = plt.figure()
        ax = plt.axes(projection = '3d')
        ax.scatter3D(self.X[:,0],self.X[:,1],self.X[:,2],c='red',label='original')
        ax.scatter3D(self.X_approx[:,0],self.X_approx[:,1],self.X_approx[:,2],c='blue',label='Decoded')
        ax.legend()

        i = 0
        for x in self.X:
            x_approx = self.X_approx[i]
            i+=1
            line = drawline(x,x_approx,axis=ax)
            line.draw()
    def draw_2D(self):
        fig2 = plt.figure()

        # plt.rcParams['legend.fontsize'] = 10
        ax = plt.axes()
        ax.scatter(self.C[:,0],self.C[:,1],c='green',label='Encoded')
        ax.legend()

In [ ]:
#Cube
X=np.array([[0,0,0],
   [0,0,1],
   [0,1,0],
   [0,1,1],
   [1,0,0],
   [1,0,1],
   [1,1,0],
   [1,1,1]])

In [ ]:
Cube = PCA(X,2)
C = Cube.Encode()
X_approx = Cube.Decode()

In [ ]:
Plot = DrawResult(X,C,X_approx)
Plot.draw_3D()
# plt.savefig("PCA_3.png")
Plot.draw_2D()
# plt.savefig("PCA_4.png")